In [1]:
# https://hotspot.readthedocs.io/en/latest/Lineage_Tutorial.html
# https://colab.research.google.com/drive/1UArV6s-3JN2nKuHcsjfsYu33elR1ySa6#scrollTo=Dj24KOME2K6l

import sys
import warnings; warnings.simplefilter('ignore')

import hotspot
import scanpy as sc

import numpy as np
import mplscience

import anndata as ad

In [2]:
adata = ad.read_h5ad("/home/users/kzlin/kzlinlab/projects/scContrastiveLearn/out/kevin/Writeup5/Larry_41093_2000_norm_log.h5ad")
adata

AnnData object with n_obs × n_vars = 41093 × 2000
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'Time point', 'clone_id', 'n_genes'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'data_des', 'hvg', 'log1p'
    obsm: 'X_clone', 'X_emb'
    layers: 'ambiguous', 'matrix', 'raw_counts', 'spliced', 'unspliced'

In [3]:
adata.obs['Cell barcode']

d6_2_2:GCGTGCAAAGAAGTTA        GCGTGCAA-AGAAGTTA
d6_2_2:AAGGGACCCTCGATGC        AAGGGACC-CTCGATGC
d6_2_2:CGTACCGAAGCGCCTT        CGTACCGA-AGCGCCTT
d6_2_2:CCGTAGCTAGGCAGTT        CCGTAGCT-AGGCAGTT
d6_2_2:TAGGCTTTCGTGGATA        TAGGCTTT-CGTGGATA
                                     ...        
LSK_d6_1_3:CTGTGACCTGCTATTT    CTGTGACC-TGCTATTT
LSK_d6_1_3:AGGCGTTTCCTATTCA    AGGCGTTT-CCTATTCA
LSK_d6_1_3:CGTGCAACTGGCCTGT    CGTGCAAC-TGGCCTGT
LSK_d6_1_3:CCCTTGCAAGTGAAAG    CCCTTGCA-AGTGAAAG
LSK_d6_1_3:TCCGCTGTGACAGATA    TCCGCTGT-GACAGATA
Name: Cell barcode, Length: 41093, dtype: category
Categories (35448, object): ['AAACAAAC-AAATCAGA', 'AAACAAAC-AAGATTGT', 'AAACAAAC-AAGCGTAC', 'AAACAAAC-AAGCTTCT', ..., 'TGTTATCA-TTGCACGC', 'TGTTATCA-TTGTCGCC', 'TGTTATCA-TTTAACAG', 'TGTTATCA-TTTGGGAG']

In [4]:
adata.layers["raw_counts"].shape

(41093, 2000)

In [5]:
scCL_embedding = np.load("/home/users/kzlin/kzlinlab/projects/scContrastiveLearn/out/kevin/Writeup5/scBaseEncoderFeat_Z_bs260_tau0.5.npy")
scCL_embedding.shape

(41093, 64)

In [6]:
adata.obsm["X_scCL"] = scCL_embedding

# Verify the assignment
print(adata.obsm["X_scCL"].shape)

(41093, 64)


Load the appropriate cells

In [36]:
import pandas as pd

# Load the trajectory file
trajectory_file = "/home/users/kzlin/kzlinlab/data/larry_hematopoiesis/stateFate_inVitro_neutrophil_monocyte_trajectory.txt"
trajectory_df = pd.read_csv(trajectory_file, sep="\t")

# Load the metadata file
metadata_file = "/home/users/kzlin/kzlinlab/data/larry_hematopoiesis/stateFate_inVitro_metadata.txt"
metadata_df = pd.read_csv(metadata_file, sep="\t")

# Filter metadata based on the cell indices in trajectory_df
filtered_metadata = metadata_df.iloc[trajectory_df['Cell index'].values]

# Concatenate 'Library' and 'Cell barcode' in filtered_metadata to create unique barcodes
filtered_metadata['combined_barcode'] = filtered_metadata['Library'] + ':' + filtered_metadata['Cell barcode']

# Update kept_cell_barcodes to use the new combined barcodes
kept_cell_barcodes = filtered_metadata['combined_barcode'].tolist()

# Optionally, print the updated list of combined barcodes
print(len(kept_cell_barcodes))



96373


In [37]:
print(metadata_df[0:10])
print(metadata_df.shape)

  Library       Cell barcode  Time point Starting population  \
0  d6_2_2  GCGTGCAA-AGAAGTTA         6.0       Lin-Kit+Sca1-   
1  d6_2_2  AAGGGACC-CTCGATGC         6.0       Lin-Kit+Sca1-   
2  d6_2_2  CGTACCGA-AGCGCCTT         6.0       Lin-Kit+Sca1-   
3  d6_2_2  CTGAAGGG-AGGAGCTT         6.0       Lin-Kit+Sca1-   
4  d6_2_2  CCGTAGCT-AGGCAGTT         6.0       Lin-Kit+Sca1-   
5  d6_2_2  GGCTTTGC-CGAACGTA         6.0       Lin-Kit+Sca1-   
6  d6_2_2  TAGGCTTT-CGTGGATA         6.0       Lin-Kit+Sca1-   
7  d6_2_2  GGCCCAAT-GTTTACGT         6.0       Lin-Kit+Sca1-   
8  d6_2_2  AAGTCGGG-GGCCTAAG         6.0       Lin-Kit+Sca1-   
9  d6_2_2  ACGGTTGG-CAGACCAT         6.0       Lin-Kit+Sca1-   

  Cell type annotation  Well  SPRING-x  SPRING-y  
0     Undifferentiated     2   411.496   -96.190  
1     Undifferentiated     2  -587.462  -306.925  
2             Monocyte     2  1429.805  -429.300  
3           Neutrophil     2  1150.028 -2030.369  
4     Undifferentiated     2 -1169.594  

In [38]:
print(filtered_metadata[0:10])

   Library       Cell barcode  Time point Starting population  \
0   d6_2_2  GCGTGCAA-AGAAGTTA         6.0       Lin-Kit+Sca1-   
2   d6_2_2  CGTACCGA-AGCGCCTT         6.0       Lin-Kit+Sca1-   
3   d6_2_2  CTGAAGGG-AGGAGCTT         6.0       Lin-Kit+Sca1-   
5   d6_2_2  GGCTTTGC-CGAACGTA         6.0       Lin-Kit+Sca1-   
6   d6_2_2  TAGGCTTT-CGTGGATA         6.0       Lin-Kit+Sca1-   
7   d6_2_2  GGCCCAAT-GTTTACGT         6.0       Lin-Kit+Sca1-   
8   d6_2_2  AAGTCGGG-GGCCTAAG         6.0       Lin-Kit+Sca1-   
10  d6_2_2  GCGTATTC-CGTGGATA         6.0       Lin-Kit+Sca1-   
11  d6_2_2  AAACACTA-CATCGCAG         6.0       Lin-Kit+Sca1-   
12  d6_2_2  TGCTTCAT-GCCTCTTT         6.0       Lin-Kit+Sca1-   

   Cell type annotation  Well  SPRING-x  SPRING-y          combined_barcode  
0      Undifferentiated     2   411.496   -96.190  d6_2_2:GCGTGCAA-AGAAGTTA  
2              Monocyte     2  1429.805  -429.300  d6_2_2:CGTACCGA-AGCGCCTT  
3            Neutrophil     2  1150.028 -2030.369 

In [39]:
filtered_metadata['Cell type annotation'].value_counts()

Cell type annotation
Undifferentiated    55570
Neutrophil          21626
Monocyte            19175
Erythroid               2
Name: count, dtype: int64

In [40]:
kept_cell_barcodes

['d6_2_2:GCGTGCAA-AGAAGTTA',
 'd6_2_2:CGTACCGA-AGCGCCTT',
 'd6_2_2:CTGAAGGG-AGGAGCTT',
 'd6_2_2:GGCTTTGC-CGAACGTA',
 'd6_2_2:TAGGCTTT-CGTGGATA',
 'd6_2_2:GGCCCAAT-GTTTACGT',
 'd6_2_2:AAGTCGGG-GGCCTAAG',
 'd6_2_2:GCGTATTC-CGTGGATA',
 'd6_2_2:AAACACTA-CATCGCAG',
 'd6_2_2:TGCTTCAT-GCCTCTTT',
 'd6_2_2:ATCCTAGT-AATCGGGT',
 'd6_2_2:AGGGACTG-ATATACCT',
 'd6_2_2:AGACGAGG-GTCTCTCT',
 'd6_2_2:GTCAGACC-AAGATTGT',
 'd6_2_2:AAACTACA-CAATACTT',
 'd6_2_2:TGCGACTA-CAACAAAT',
 'd6_2_2:TAGGTACG-ACCTTGCC',
 'd6_2_2:GAAGATAT-TGGAGGAG',
 'd6_2_2:GGATTGTT-CTCTTGAC',
 'd6_2_2:ATCCCACC-GGTTTCTC',
 'd6_2_2:TACCCTGC-CGCCTTGA',
 'd6_2_2:ATGACTTG-AGACACCA',
 'd6_2_2:TGCCGGTA-TCCCTGAT',
 'd6_2_2:GGCAAGGT-GCAAAGCC',
 'd6_2_2:ACTACCCG-GTCAATAC',
 'd6_2_2:CCTTTACA-CTTCTTCG',
 'd6_2_2:AATATGAC-GTTTACGT',
 'd6_2_2:TAACTTCT-AACCTGAC',
 'd6_2_2:GGTTGAGA-GAGCCCAT',
 'd6_2_2:ATACACCC-ATCAGCGC',
 'd6_2_2:CTTAGTGT-TCCAGTCC',
 'd6_2_2:TGGTGTCT-CATACATT',
 'd6_2_2:CTGCGATG-GGCATGCT',
 'd6_2_2:TCGTTCCC-TCATCCTT',
 'd6_2_2:GATAC

In [41]:
print(trajectory_df[0:10])

   Cell index
0           0
1           2
2           3
3           5
4           6
5           7
6           8
7          10
8          11
9          12


In [18]:
len(kept_cell_barcodes)

96373

Filter cells

In [46]:
# Convert the 'Library' and 'Cell barcode' columns to string before concatenating
adata.obs['Library_str'] = adata.obs['Library'].astype(str)
adata.obs['Cell barcode_str'] = adata.obs['Cell barcode'].astype(str)

# Ensure 'adata' also has 'Library' and 'Cell barcode' columns in the same format
# Create a new 'combined_barcode' column in adata
adata.obs['combined_barcode'] = adata.obs['Library_str'] + ':' + adata.obs['Cell barcode_str']

In [47]:

# Filter the AnnData object based on the combined barcodes
filtered_adata = adata[adata.obs['combined_barcode'].isin(kept_cell_barcodes), :]

# Optionally, drop the 'combined_barcode' column after filtering if not needed anymore
filtered_adata.obs = filtered_adata.obs.drop(columns=['combined_barcode'])

# Print the filtered AnnData object
print(filtered_adata)


AnnData object with n_obs × n_vars = 29137 × 2000
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'Time point', 'clone_id', 'n_genes', 'Library_str', 'Cell barcode_str'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'data_des', 'hvg', 'log1p'
    obsm: 'X_clone', 'X_emb', 'X_scCL'
    layers: 'ambiguous', 'matrix', 'raw_counts', 'spliced', 'unspliced'


In [48]:
filtered_adata.obs['state_info'].value_counts()

state_info
Undifferentiated    14360
Neutrophil           7427
Monocyte             7349
Erythroid               1
Name: count, dtype: int64

In [49]:
filtered_adata.obs['time_info'].value_counts()

time_info
6.0    18511
4.0     9220
2.0     1406
Name: count, dtype: int64

In [53]:
# Filter the filtered_adata based on 'time_info' being equal to 2
filtered_adata_time2 = filtered_adata[filtered_adata.obs['time_info'] == 2, :]

# Print or inspect the new filtered AnnData object
print(filtered_adata_time2)

View of AnnData object with n_obs × n_vars = 1406 × 2000
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'Time point', 'clone_id', 'n_genes', 'Library_str', 'Cell barcode_str'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'data_des', 'hvg', 'log1p'
    obsm: 'X_clone', 'X_emb', 'X_scCL'
    layers: 'ambiguous', 'matrix', 'raw_counts', 'spliced', 'unspliced'


In [55]:
filtered_adata_time2.obs['state_info'].value_counts()

state_info
Undifferentiated    1355
Monocyte              32
Neutrophil            19
Name: count, dtype: int64

Run Hotspot

In [56]:
sc.pp.calculate_qc_metrics(filtered_adata_time2, inplace=True)
sc.pp.filter_genes(filtered_adata_time2, min_cells=1)

In [58]:
filtered_adata_time2.layers["raw_counts_csc"] = filtered_adata_time2.layers["raw_counts"].tocsc()

In [59]:
# Create the Hotspot object with latent embeddings and normalized data

hs = hotspot.Hotspot(
    filtered_adata_time2,
    layer_key="raw_counts_csc",
    model='danb',
    latent_obsm_key="X_scCL"
)

In [60]:
# Create the neighborhood graph
hs.create_knn_graph(
    weighted_graph=False, n_neighbors=30,
)

In [61]:
# Determine informative genes
hs_results = hs.compute_autocorrelations(jobs=4)
print(hs_results.head(15))

100%|██████████| 1548/1548 [00:01<00:00, 1349.45it/s]

                C          Z           Pval            FDR
Gene                                                      
Ly6a     0.266491  84.563103   0.000000e+00   0.000000e+00
Dlk1     0.220047  60.513222   0.000000e+00   0.000000e+00
Prtn3    0.551025  54.787540   0.000000e+00   0.000000e+00
Mpo      0.523665  51.689915   0.000000e+00   0.000000e+00
Gata2    0.183826  48.563732   0.000000e+00   0.000000e+00
Gzmb     0.089298  46.728737   0.000000e+00   0.000000e+00
Akr1c13  0.156962  40.835007   0.000000e+00   0.000000e+00
Elane    0.563921  40.494400   0.000000e+00   0.000000e+00
Ctsg     0.451196  40.132843   0.000000e+00   0.000000e+00
Casp6    0.172298  36.058033  5.160555e-285  7.988539e-283
H2afy    0.189708  35.803814  4.816721e-281  6.778441e-279
Ctla2a   0.152750  34.291538  5.246026e-258  6.767374e-256
Muc13    0.127593  31.408346  7.782184e-217  9.266786e-215
Tsc22d1  0.127888  30.565125  1.780083e-205  1.968263e-203
Pim2     0.121126  29.253271  2.040036e-188  2.105317e-1

In [62]:
print(hs_results.tail(15))

                      C         Z      Pval       FDR
Gene                                                 
2310001H17Rik -0.009361 -1.521156  0.935890  0.944431
Fgfr1         -0.005733 -1.536094  0.937742  0.945103
Armcx6        -0.010358 -1.536375  0.937777  0.945103
Rgs2          -0.009874 -1.549452  0.939363  0.946086
Cyp4v3        -0.005795 -1.595884  0.944743  0.950769
Wtip          -0.007992 -1.600366  0.945241  0.950769
Dopey2        -0.010319 -1.606169  0.945882  0.950795
Tuft1         -0.007548 -1.649287  0.950456  0.954773
Rsad1         -0.007690 -1.682079  0.953723  0.957434
Tmem37        -0.006273 -1.770932  0.961714  0.964830
Dennd1a       -0.010016 -1.815754  0.965296  0.967797
Adora2a       -0.007710 -1.841624  0.967235  0.969113
Trp53inp1     -0.012606 -1.952523  0.974562  0.975823
Prune2        -0.011659 -1.996498  0.977060  0.977692
Mctp2         -0.007222 -2.386420  0.991493  0.991493


In [63]:
import pandas as pd

# Assuming df is your DataFrame
hs_results.to_csv('/home/users/kzlin/kzlinlab/projects/scContrastiveLearn/git/SCSeq_LineageBarcoding_kevin/csv/kevin/Writeup5/Writeup5_LARRY_LCL_hotspot_day2_autocorrelations.csv')  